# Matrix-matrix multiplication by columns

We now look at how the FLAMEJulia API can be used to implement different matrix-matrix multiplication algorithms.  

First, we create some matrices.

In [27]:
m = 4
n = 3
k = 5

C = rand(m, n)
println( "C = ")
C

C = 


4×3 Array{Float64,2}:
 0.74971    0.745538  0.090947
 0.910935   0.248869  0.720639
 0.597413   0.664635  0.166778
 0.0949942  0.592252  0.724036

In [28]:
Cold = copy(C)          # an alternative way of doing a "hard" copy, in this case of a matrix
    
A = rand(m, k)
println("A = " )
A

A = 


4×5 Array{Float64,2}:
 0.637571  0.696629  0.217307  0.127578  0.102013
 0.622843  0.945459  0.899098  0.390123  0.484672
 0.266225  0.18644   0.171075  0.517504  0.726257
 0.967062  0.9392    0.912189  0.237931  0.47508 

In [29]:
B = rand(k, n)
println( "B = " )
B

B = 


5×3 Array{Float64,2}:
 0.782818  0.845178  0.784613
 0.268421  0.649096  0.507649
 0.992778  0.633345  0.278936
 0.383018  0.234097  0.86429 
 0.750724  0.243729  0.833391

## <h2>The algorithm  </h2>  

<img src="https://studio.edx.org/c4x/UTAustinX/UT.5.01x/asset/Gemm_nn_unb_var1.png" alt="Matrix-matrix multiplication by columns picture" width="80%">

<h2> The routine <code> Gemm_nn_unb_var1!( A, B, C ) </code> </h2>

This routine computes $ C := A B + C $ by columns.  The "\_nn\_" means that this is the "No transpose, No transpose" case of matrix multiplication.  
The reason for this is that the operations $ C := A^T B + C $ ("\_tn\_" or "Transpose, No transpose"), $ C := A B^T + C $ ("\_nt\_" or "No transpose, Transpose"), and $ C := A^T B^T + C $ ("\_tt\_" or "Transpose, Transpose") are also encountered.  
    
The specific laff function we will use is
<ul>
<li> <code> laff.gemv!( trans, alpha, A, x, beta, y ) </code> which computes 
$ y := \alpha A x + \beta y $ or $ y := \alpha A^T x + \beta y $, depending on 
        parameter <code> trans</code>.  In particular, 
        <ul>
        <li>
        <code> laff.gemv!( "No transpose", alpha, A, x, beta, y ) </code> computes $ y := \alpha A x + \beta y $.
            </li>
        <li>
        <code> laff.gemv!( "Transpose", alpha, A, x, beta, y ) </code> computes $ y := \alpha A^T x + \beta y $.
            </li>
            </ul>
    </li>
</ul>

Use the <a href="https://studio.edx.org/c4x/UTAustinX/UT.5.01x/asset/index.html"> Spark webpage</a> to generate a code skeleton.  (Make sure you adjust the name of the routine.)

In [53]:
include("../flame.jl")
using .flame
include("../laff/laff.jl")
using .laff

function Gemm_nn_unb_var1!(A, B, C)

    BL, BR = flame.part_1x2(B, 
                            0, "LEFT")

    CL, CR = flame.part_1x2(C, 
                            0, "LEFT")

    while size(BL, 2) < size(B, 2)

        B0, b1, B2 = flame.repart_1x2_to_1x3(BL, BR, 
                                             1, "RIGHT")

        C0, c1, C2 = flame.repart_1x2_to_1x3(CL, CR, 
                                             1, "RIGHT")

        #------------------------------------------------------------#

        laff.gemv!( "No transpose", 1.0, A, b1, 1.0, c1 )

        #------------------------------------------------------------#

        BL, BR = flame.cont_with_1x3_to_1x2(B0, b1, B2, 
                                            "LEFT")

        CL, CR = flame.cont_with_1x3_to_1x2(C0, c1, C2, 
                                            "LEFT")

    end

    flame.merge_1x2!(CL, CR, C)

end

Gemm_nn_unb_var1! (generic function with 1 method)

In [54]:
C = copy( Cold ) # restore C 

Gemm_nn_unb_var1!( A, B, C )

println( "C - ( Cold + A * B )" )
C - ( Cold + A * B )

C - ( Cold + A * B )


4×3 Array{Float64,2}:
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0

Bingo! It works!

## Watch the algorithm at work!

Copy and paste the code into <a href="http://edx-org-utaustinx.s3.amazonaws.com/UT501x/PictureFlame/PictureFLAME.html"> PictureFLAME </a>, a webpage where you can watch your routine in action.  Just cut and paste into the box.  

Disclaimer: we implemented a VERY simple interpreter.  If you do something wrong, we cannot guarantee the results.  But if you do it right, you are in for a treat.

If you want to reset the problem, just click in the box into which you pasted the code and hit "next" again.